# ezlocalai Tests and Examples

Simply choose your favorite model of choice from the models list and paste it into the `model` variable on the API calls. You can get a list of models below.

Install OpenAI and requests:

```bash
pip install openai requests python-dotenv
```

**Note, you do not need an OpenAI API Key, the API Key is your `EZLOCALAI_API_KEY` for the server if you defined one in your `.env` file.**

## Global definitions and helpers

Confirm that your `DEFAULT_MODEL` is set to the model you want to use in your `.env` file.


In [1]:
import openai
import requests
import time
import os
import re
from dotenv import load_dotenv

load_dotenv()

# Set your system message, max tokens, temperature, and top p here, or use the defaults.
SYSTEM_MESSAGE = "The assistant is acting as a creative writer. All of your text responses are transcribed to audio and sent to the user. Be concise with all responses. After the request is fulfilled, end with </s>."
DEFAULT_MAX_TOKENS = 512
DEFAULT_TEMPERATURE = 0.5
DEFAULT_TOP_P = 0.9

# ------------------- DO NOT EDIT BELOW THIS LINE IN THIS CELL ------------------- #
EZLOCALAI_SERVER = os.getenv("EZLOCALAI_SERVER", "http://localhost:8091")
EZLOCALAI_API_KEY = os.getenv("EZLOCALAI_API_KEY", "none")
DEFAULT_LLM = os.getenv("DEFAULT_LLM", "phi-2-dpo")
openai.base_url = f"{EZLOCALAI_SERVER}/v1/"
openai.api_key = EZLOCALAI_API_KEY if EZLOCALAI_API_KEY else EZLOCALAI_SERVER
HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"{EZLOCALAI_API_KEY}",
    "ngrok-skip-browser-warning": "true",
}


def display_content(content):
    global EZLOCALAI_SERVER
    global HEADERS
    outputs_url = f"{EZLOCALAI_SERVER}/outputs/"
    os.makedirs("outputs", exist_ok=True)
    try:
        from IPython.display import Audio, display, Image, Video
    except:
        print(content)
        return
    if "http://localhost:8091/outputs/" in content:
        if outputs_url != "http://localhost:8091/outputs/":
            content = content.replace("http://localhost:8091/outputs/", outputs_url)
    if "<audio controls>" in content or " " not in content:
        import base64
        from datetime import datetime

        try:
            audio_response = content.split("data:audio/wav;base64,")[1].split('" type')[
                0
            ]
        except:
            audio_response = content
        file_name = f"outputs/{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}.wav"
        with open(file_name, "wb") as fh:
            fh.write(base64.b64decode(audio_response))
        display(Audio(filename=file_name, autoplay=True))
    if outputs_url in content:
        urls = re.findall(f"{re.escape(outputs_url)}[^\"' ]+", content)
        urls = urls[0].split("\n\n")
        for url in urls:
            file_name = url.split("/")[-1]
            url = f"{outputs_url}{file_name}"
            data = requests.get(url, headers=HEADERS).content
            if url.endswith(".jpg") or url.endswith(".png"):
                content = content.replace(url, "")
                display(Image(data=data))
            elif url.endswith(".mp4"):
                content = content.replace(url, "")
                display(Video(data=data, autoplay=True))
            elif url.endswith(".wav"):
                content = content.replace(url, "")
                display(Audio(data=data, autoplay=True))
    print(content)

## Language Models

Get a list of models to choose from if you don't already know what model you want to use.


In [2]:
# Wait for server to come up instead of timing out.
while True:
    try:
        models = requests.get(f"{EZLOCALAI_SERVER}/v1/models", headers=HEADERS)
        if models.status_code == 200:
            break
    except:
        pass
    time.sleep(1)
print(models.json())

['bakllava-1-7b', 'llava-v1.5-7b', 'llava-v1.5-13b', 'yi-vl-6b', 'CapybaraHermes-2.5-Mistral-7B', 'CapybaraHermes-2.5-Mistral-7B', 'KafkaLM-70B-German-V0.1', 'CodeLlama-70B-Python', 'CodeLlama-70B-Instruct', 'CodeLlama-70B-hf', 'Tess-34B-v1.5b', 'Tess-10.7B-v1.5b', 'Goliath-longLORA-120b-rope8-32k-fp16', 'Etheria-55b-v0.1', 'EstopianMaid-13B', 'Everyone-Coder-33B-Base', 'FusionNet_34Bx2_MoE', 'WestLake-7B-v2', 'WestSeverus-7B-DPO', 'DiscoLM_German_7b_v1', 'Garrulus', 'DareVox-7B', 'NexoNimbus-7B', 'Lelantos-Maid-DPO-7B', 'stable-code-3b', 'Dr_Samantha-7B', 'NeuralBeagle14-7B', 'tigerbot-13B-chat-v5', 'Nous-Hermes-2-Mixtral-8x7B-SFT', 'Thespis-13B-DPO-v0.7', 'Code-290k-13B', 'Nous-Hermes-2-Mixtral-8x7B-DPO', 'Venus-120b-v1.2', 'LLaMA2-13B-Estopia', 'medicine-LLM', 'finance-LLM-13B', 'Yi-34B-200K-DARE-megamerge-v8', 'phi-2-orange', 'laser-dolphin-mixtral-2x7b-dpo', 'bagel-dpo-8x7b-v0.2', 'Everyone-Coder-4x7b-Base', 'phi-2-electrical-engineering', 'Cosmosis-3x34B', 'HamSter-0.1', 'Helion-

## Voices

Any `wav` file in the `voices` directory will be available to use as a voice.


In [3]:
voices = requests.get(f"{EZLOCALAI_SERVER}/v1/audio/voices", headers=HEADERS)
print(voices.json())

{'voices': ['default', 'DukeNukem', 'Hal9000_Mono', 'Hal_voice_9000_Synthetic', 'SyntheticStarTrekComputerVoice', 'Synthetic_DukeNukem', 'Synthetic_Female_Hybrid_4_Phonetics_0001', 'Synthetic_Female_Phonetics_0001']}


## Embeddings

[OpenAI API Reference](https://platform.openai.com/docs/api-reference/embeddings)


In [4]:
# Modify this prompt to generate different outputs
prompt = "Tacos are great."

response = openai.embeddings.create(
    input=prompt,
    model=DEFAULT_LLM,
)
print(response.data[0].embedding)

[0.004453756578405207, 0.017969004307333933, 0.03606555641227144, -0.0054616568378164755, 0.015536065215060135, -0.003803916527160804, 0.015357981031897238, 0.020200151120728423, -0.013636552896760513, 0.023346456251072553, 0.008404845399992549, -0.0028292180920603477, 0.009283579933552753, 0.001405623021436959, -0.005578390203966923, -0.0058644958664325056, 0.012267600898233214, 0.02614486690917118, 0.0008285432858533403, -0.006195183543999331, -0.03189215331086669, -0.013050255867884683, -0.005148174161713314, -0.007974882649359841, 0.010753003976168601, 0.01980962654792576, 0.007285445272452794, 0.014627283570704669, 0.0039605915162568206, -0.007206821760109855, -0.011690170421000208, -0.01613375633214276, 0.0006373446488284059, 0.007290331783369307, 0.01100753560525345, -0.005880943682855987, -0.012475815576689581, -0.008465660494019197, 0.020869321581533297, -0.02970492053998027, 0.006063917515061673, 0.01035485196329864, 0.011749383277410746, 0.016813574903708096, -0.008306176433

## Chat Completion

[OpenAI API Reference](https://platform.openai.com/docs/api-reference/chat)


In [5]:
# Modify this prompt to generate different outputs
prompt = "Write a short poem about Pikachu with a picture."


response = openai.chat.completions.create(
    model=DEFAULT_LLM,
    messages=[{"role": "user", "content": prompt}],
    temperature=DEFAULT_TEMPERATURE,
    max_tokens=DEFAULT_MAX_TOKENS,
    top_p=DEFAULT_TOP_P,
    stream=False,
    extra_body={"system_message": SYSTEM_MESSAGE},
)
display_content(response.messages[1]["content"])

In the lush green forests of Kanto,
  A tiny creature roams with boundless glee,
  A yellow ball of fur, so bright and bold,
  Pikachu, the electric friend so bold.

  With lightning-fast speed, it darts through the trees,
  A smile on its face, its heart full of ease,
  A loyal companion, by its side it stays,
  Through battles fierce, and adventures that lay.

  Its fur glistens in the sun's golden rays,
  A sparkle in its eyes, like stars in the night's haze,
  A friend so pure, so full of love,
  Pikachu, the heart of Kanto's people.

  So let us celebrate this little friend so bright,
  For in its presence, our hearts take flight,
  For in its laughter, we find joy anew,
  For in its love, we find what we're meant to do.

  Pikachu, the heart of Kanto's people,
  Forever in our memories, our hearts will be free.


## Completion

[OpenAI API Reference](https://platform.openai.com/docs/api-reference/completions/create)


In [6]:
# Modify this prompt to generate different outputs
prompt = "Write a haiku about the future."

completion = openai.completions.create(
    model=DEFAULT_LLM,
    prompt=prompt,
    temperature=DEFAULT_TEMPERATURE,
    max_tokens=DEFAULT_MAX_TOKENS,
    top_p=DEFAULT_TOP_P,
    n=1,
    stream=False,
    extra_body={"system_message": SYSTEM_MESSAGE},
)
display_content(completion.choices[0].text)

A world in flux,
  Emerging technologies rise,
  Hope for tomorrow.

This haiku captures the essence of the rapid changes in our world due to advancing technology, while also expressing optimism for the future. The first line sets the scene, describing a world in flux or constantly changing. The second line introduces the rising technologies that are shaping our present and future. Finally, the third line offers a hopeful message, emphasizing that there is still reason for optimism in the face of these changes.


## Cloning Text to Speech

Any `wav` file in the `voices` directory can be used as a voice.


In [28]:
from pathlib import Path
import base64
import IPython.display as ipd

prompt = "Write a short poem about vikings with a picture."
os.makedirs("outputs", exist_ok=True)
speech_file_path = Path("outputs/speech.wav")
tts_response = openai.audio.speech.create(
    model="tts-1",
    voice="DukeNukem",
    input=prompt,
    extra_body={"language": "en"},
)
audio_content = base64.b64decode(tts_response.content)
speech_file_path.write_bytes(audio_content)

ipd.Audio(speech_file_path)

## Text to Speech


In [22]:
import uuid
import os

os.makedirs("outputs", exist_ok=True)
audio_path = os.path.join(os.getcwd(), "outputs", f"{uuid.uuid4()}.wav")

with open(audio_path, "wb") as audio_file:
    audio_file.write(audio_content)

with open(audio_path, "rb") as audio_file:
    transcription = openai.audio.transcriptions.create(model="base", file=audio_file)

print(transcription.text)

 Write a short poem about Vikings with a picture.


## Voice Completion Example


In [30]:
# We will use the audio response from a couple of cells back.
completion = openai.completions.create(
    model=DEFAULT_LLM,
    prompt=tts_response.content.decode("utf-8"),
    temperature=DEFAULT_TEMPERATURE,
    max_tokens=DEFAULT_MAX_TOKENS,
    top_p=DEFAULT_TOP_P,
    n=1,
    stream=False,
    extra_body={
        "system_message": SYSTEM_MESSAGE,
        "audio_format": "wav",
        "voice": "DukeNukem",
    },
)

response_text = completion.choices[0].text
display_content(response_text)

In the land of ice and fire, where legends were born,
  The mighty Vikings sailed across the seas, fierce and strong.
  Their ships, like giants on water, they'd set sail,
  To conquer new lands, their courage never pale.

  With swords sharp as lightning, they'd battle foes unseen,
  Their fierce warriors fought for honor, their valor never dimmed.
  Their longships, sleek and swift, they'd navigate the waves,
  Their eyes set on glory, their hearts ablaze.

  In the land of the Northern Lights, they'd build their homes,
  Their longhouses, like castles, stood tall against the storm.
  Their families strong, their bonds unbreakable,
  Their traditions passed down, their legacy everlasting.

  And when night fell, they'd gather around the fire,
  Their stories told, their songs sung, their spirits never tire.
  For in their hearts, they'd always be true,
  To their land, their people, their Viking ways anew.

  So let us remember them, in our hearts and minds,
  For they were fierce an